## Dataset Preparation

In [1]:
from pathlib import Path
import pandas as pd

region = 'SA1'

def load_nem_data(table, region=None, region_col='REGIONID'):
    anchor = Path.home() / 'nem-data' / 'data' / table
    clean = anchor.glob('**/clean.parquet')
    fis = [pd.read_parquet(f) for f in clean]
    data = pd.concat(fis, axis=0)

    if region:
        mask = data[region_col] == region
        data = data.loc[mask, :]
    return data

In [ ]:
prices = load_nem_data('TRADINGPRICE', region)
prices = prices.drop_duplicates().set_index('interval-start')
prices = prices.resample('5T').ffill()
prices.head(2)

In [ ]:
def nem_sites():
    url = (
        "http://www.nemweb.com.au/Reports/CURRENT/CDEII/CO2EII_AVAILABLE_GENERATORS.CSV"
    )
    print(f"nem_sites, {url}")

    raw = pd.read_csv(url)
    sites = pd.read_csv(url, skiprows=1).iloc[:-1, :]
    sites = sites.rename(
        {
            "DUID": "site",
            "CO2E_EMISSIONS_FACTOR": "carbon_intensity_[tc/mwh]",
            "CO2E_ENERGY_SOURCE": "energy_source",
        },
        axis=1,
    )
    return sites
sites = nem_sites()
sites.head(2)

## Carbon Data

Carbon data needs a bit of work - select only ENOF rows:

In [ ]:
carbon = load_nem_data('nemde', region, 'RegionID')

In [ ]:
carbon['PeriodID'] = pd.to_datetime(carbon['PeriodID'])
carbon['PeriodID'] = carbon['PeriodID'].dt.tz_localize(None)
carbon

In [ ]:
mask = carbon['Market'] == 'Energy'
carbon = carbon.loc[mask, :]

mask = carbon['DispatchedMarket'] == 'ENOF'
carbon = carbon.loc[mask, :]

carbon['PeriodID'] = pd.to_datetime(carbon['PeriodID'])

Want to join on NEM Sites

In [ ]:
sites.head(2)

In [ ]:
carbon.head(2)

In [ ]:
carbon = carbon.merge(sites[['site', 'REGIONID', 'carbon_intensity_[tc/mwh]']], left_on='Unit', right_on='site', how='left')

In [ ]:
mask = carbon['carbon_intensity_[tc/mwh]'].isnull()
carbon = carbon.loc[~mask, :]
assert carbon.isnull().sum().sum() == 0

In [ ]:
carbon['carbon_generation_[tc/h]'] = carbon['carbon_intensity_[tc/mwh]'] * carbon['Increase']

carbon = carbon.set_index('PeriodID')

In [ ]:
grp = carbon.groupby(carbon.index).agg({
    'carbon_generation_[tc/h]': 'sum',
    'Increase': 'sum',
})

In [ ]:
grp.sort_values('Increase').head(4)

In [ ]:
carbon.head(2)

Drop wind for coal:

In [ ]:
carbon.loc[
    '2014-01-01 12:00:00', :
]

In [ ]:
ds = prices.merge(carbon, how='left', left_index=True, right_index=True).sort_index()
print(ds.columns)

ds = ds.loc[:,[
    'RRP', 'Increase'
]]
ds.head(3)

In [ ]:
ds = ds.dropna(axis=0)
assert ds.isnull().sum().sum() == 0

In [ ]:
ds.shape

## Battery Modelling

In [ ]:
!pip install -q pandas==1.3.3 fastparquet
import energypylinear as epl

In [ ]:
mdl = epl.Battery(power=2, capacity=4)
mdl

Want to see profit versus carbon per year

Also look at correlation between price & average carbon

In [ ]:
subset.head(3)

In [ ]:
subset = ds.iloc[:1000, :]

opt_price = mdl.optimize(
    prices=subset['RRP'], carbon=subset['Increase'], objective='price'
)

In [ ]:
carbon_opt = mdl.optimize(
    prices=subset['RRP'], carbon=subset['Increase'], objective='carbon'
)

In [ ]:
opt_price = pd.DataFrame(opt_price)
cols = [c for c in opt_price.columns if 'Cost' in c]
opt_price.sum()[cols]

In [ ]:
pd.DataFrame(carbon_opt).sum()[cols]

In [ ]:
cols